In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
%cd rapidsai-csp-utils
!git checkout patch-22.12
%cd ..
!python rapidsai-csp-utils/colab/env-check.py
!python rapidsai-csp-utils/colab/pip-install.py

# Install the Merlin Framework
!pip install -U git+https://github.com/NVIDIA-Merlin/models.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/nvtabular.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/core.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/system.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/dataloader.git@release-23.04
!pip install -U git+https://github.com/NVIDIA-Merlin/Transformers4Rec.git@release-23.04
!pip install -U xgboost lightfm implicit


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 8.24 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/rapidsai-csp-utils
error: pathspec 'patch-22.12' did not match any file(s) known to git
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************

***********************************************************************
Woo! Your instance 

In [3]:
import os
import pandas as pd
import nvtabular as nvt
from merlin.models.utils.example_utils import workflow_fit_transform
import merlin.io

import merlin.models.tf as mm

from nvtabular import ops
from merlin.core.utils import download_file
from merlin.datasets.entertainment import get_movielens
from merlin.schema.tags import Tags

/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


We will use the utils function to download, extract and preprocess the dataset.

In [4]:
input_path = os.environ.get("INPUT_DATA_DIR", os.path.expanduser("~/merlin-models-data/movielens/"))
train, valid = get_movielens(variant="ml-1m", path=input_path)

downloading ml-1m.zip: 5.93MB [00:00, 13.9MB/s]                            
unzipping files: 100%|██████████| 5/5 [00:00<00:00, 21.82files/s]
/usr/local/lib/python3.10/dist-packages/merlin/datasets/entertainment/movielens/dataset.py:227: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/merlin/datasets/entertainment/movielens/dataset.py:232: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/merlin/datasets/entertainment/movielens/dataset.py:237: ParserWarning: Falling back to the 'python' 

In [5]:
data = get_movielens(variant="ml-1m", path=input_path)

# Accessing the first element of the tuple
features = data[0]

# Converting the Dataset object to a DataFrame
features_df = features.to_ddf().compute()

# Printing the first row of the DataFrame
#print('Features:', features_df.iloc[0])
features_df.iloc[0:10]

,userId,movieId,title,genres,gender,age,occupation,zipcode,TE_age_rating,TE_gender_rating,TE_occupation_rating,TE_zipcode_rating,TE_movieId_rating,TE_userId_rating,rating_binary,rating
0,1720,1404,1404,"[1, 6]",2,1,8,1273,-0.530049,1.726670,-0.208687,0.556522,-0.662098,0.501847,1,4.0
1,2774,7,7,"[3, 5, 4]",1,7,9,874,-0.424362,-0.557833,0.693842,-0.426427,0.942323,-0.156971,1,5.0
2,910,23,23,"[3, 7, 15, 5]",1,1,8,828,-0.505215,-0.544541,-0.250274,0.713782,-0.416747,0.473308,0,1.0
3,900,986,983,"[5, 4]",1,3,9,793,-1.060359,-0.583126,0.698297,0.124422,-2.076140,-0.126654,1,4.0
4,2819,75,75,[2],2,1,9,127,-0.539711,1.751015,0.693842,-0.485308,0.954622,0.189941,1,4.0
5,1045,26,26,"[3, 8, 2]",1,2,11,51,0.539091,-0.584343,1.351889,-0.237410,1.858759,-0.538939,1,4.0
6,549,809,809,"[3, 4]",1,3,12,623,-1.092628,-0.593251,1.281562,-0.283621,-0.946918,-0.272289,1,4.0
7,121,1139,1140,"[3, 2]",1,3,6,86,-1.077875,-0.544541,-1.443869,-0.313805,-2.694657,-0.347332,0,2.0
8,1410,73,73,"[7, 2, 16]",1,1,7,1333,-0.525743,-0.583126,1.260734,0.596032,0.662618,0.532371,1,4.0
9,275,216,216,"[3, 6]",2,1,11,379,-0.539711,1.751015,1.455064,-0.871763,0.155788,-0.903714,1,4.0


P.s. You can also choose to generate synthetic data to test your models using `generate_data()`. The `input` argument can be either the name of one of the supported public datasets (e.g. "movielens-1m", "criteo") or the schema of a dataset (which is explained next). For example:

```python
from merlin.datasets.synthetic import generate_data
train, valid = generate_data(input="movielens-1m", num_rows=1000000, set_sizes=(0.8, 0.2))
```

## Understanding the Schema File and Structure

The `schema` can be interpreted as a list of features in the dataset, where each element describes metadata of the feature. It contains the name, some properties (e.g. statistics) depending on the feature type and multiple tags.

In [6]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.start_index,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,userId,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.userId.parquet,0.0,210.0,6041.0,0.0,6040.0,userId,NaN,NaN
1,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID, Tags.IT...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.movieId.parquet,0.0,159.0,3685.0,0.0,3684.0,movieId,NaN,NaN
2,title,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.title.parquet,0.0,159.0,3685.0,0.0,3684.0,title,NaN,NaN
3,genres,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",True,True,0.0,NaN,0.0,.//categories/unique.genres.parquet,0.0,16.0,19.0,0.0,18.0,genres,1.0,6.0
4,gender,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.gender.parquet,0.0,16.0,3.0,0.0,2.0,gender,NaN,NaN
5,age,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.age.parquet,0.0,16.0,8.0,0.0,7.0,age,NaN,NaN
6,occupation,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.occupation.parquet,0.0,16.0,22.0,0.0,21.0,occupation,NaN,NaN
7,zipcode,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,.//categories/unique.zipcode.parquet,0.0,153.0,3440.0,0.0,3439.0,zipcode,NaN,NaN
8,TE_age_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TE_gender_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can select the features by **name**.

In [7]:
train.schema.select_by_name("userId")

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.start_index,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name
0,userId,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,0.0,None,0.0,.//categories/unique.userId.parquet,0.0,210.0,6041.0,0,6040,userId


We can also select features by **tags**. As we described earlier in the notebook, categorical and continuous features are automatically tagged when using ops like `Categorify()` and `Normalize()`.
In our example preprocessing workflow for this dataset, we also set the `Tags` for the the `user` and `item` features, and also for the `user_id` and `item_id`, which are important for collaborative filtering architectures.

Alternatively, we can select them by `Tag`. We add `column_names` to the object to receive only names without all the additional metadata.

In [8]:
# All categorical features
train.schema.select_by_tag(Tags.CATEGORICAL).column_names

['userId',
 'movieId',
 'title',
 'genres',
 'gender',
 'age',
 'occupation',
 'zipcode']

In [9]:
# All continuous features
train.schema.select_by_tag(Tags.CONTINUOUS).column_names

['TE_age_rating',
 'TE_gender_rating',
 'TE_occupation_rating',
 'TE_zipcode_rating',
 'TE_movieId_rating',
 'TE_userId_rating']

In [10]:
# All targets
train.schema.select_by_tag(Tags.TARGET).column_names

['rating_binary', 'rating']

In [11]:
# All features related to the item
train.schema.select_by_tag(Tags.ITEM).column_names

['movieId', 'genres', 'TE_movieId_rating']

In [12]:
# The item id feature name
train.schema.select_by_tag(Tags.ITEM_ID).column_names

['movieId']

In [13]:
# All features related to the user
train.schema.select_by_tag(Tags.USER).column_names

['userId',
 'TE_age_rating',
 'TE_gender_rating',
 'TE_occupation_rating',
 'TE_zipcode_rating',
 'TE_userId_rating']

In [14]:
# The user id feature name
train.schema.select_by_tag(Tags.USER_ID).column_names

['userId']

We can also query all properties of a feature. Here we see that the cardinality (number of unique values) of the `movieId` feature is `3682`, which is an important information to build the corresponding embedding table.

In [15]:
train.schema.select_by_tag(Tags.ITEM_ID)

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.start_index,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name
0,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID, Tags.IT...","DType(name='int32', element_type=<ElementType....",False,False,0.0,None,0.0,.//categories/unique.movieId.parquet,0.0,159.0,3685.0,0,3684,movieId


The `schema` is a great interface between feature engineering and modeling libraries, describing the available features and their metadata/statistics. It makes it easy to build generic models definition, as the features names and types are automatically inferred from schema and represented properly in the neural networks architectures. That means that when the dataset changes (e.g. features are added or removed), you don't have to change the modeling code to leverage the new dataset!

For example, the `DLRMModel` embeds categorical features and applies an MLP (called bottom MLP) to combine the continuous features. As another example, The `TwoTowerModel` (for retrieval) builds one MLP tower to combine user features and another MLP tower for the item features, factorizing both towers in the output.

### Download and prepare the data

We will download the dataset, if it is not already downloaded and cached locally.

In [16]:
name = "ml-1m"
download_file(
    "http://files.grouplens.org/datasets/movielens/ml-1m.zip",
    os.path.join(input_path, "ml-1m.zip"),
    redownload=False,
)

unzipping files: 100%|██████████| 5/5 [00:00<00:00, 31.58files/s]


We preprocess the dataset and split it into training and validation.

In [17]:
ratings = pd.read_csv(
    os.path.join(input_path, "ml-1m/ratings.dat"),
    sep="::",
    names=["userId", "movieId", "rating", "timestamp"],
)
# Shuffling rows
ratings = ratings.sample(len(ratings), replace=False)

num_valid = int(len(ratings) * 0.2)
train = ratings[:-num_valid]
valid = ratings[-num_valid:]
train.to_parquet(os.path.join(input_path, name, "train.parquet"))
valid.to_parquet(os.path.join(input_path, name, "valid.parquet"))

<ipython-input-17-0b2eca7a2ff7>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(


### Feature Engineering and Generating Schema File with NVTabular

We use NVTabular to define a preprocessing and feature engineering pipeline.

NVTabular has already implemented multiple transformations, called `ops` that can be applied to a `ColumnGroup` from an overloaded `>>` operator.<br><br>
**Example:**<br>
```python
features = [ column_name, ...] >> op1 >> op2 >> ...
```

We need to perform following steps:
- Categorify userId and movieId, that the values are contiguous integers from 0 ... |C|
- Transform the rating column ([1,5] interval) to a binary target by using as threshold the value `3`
- Add Tags with `ops.AddMetadata` for `item_id`, `user_id`, `item`, `user` and `target`.

Categorify will transform categorical columns into contiguous integers (`0, ..., |C|`) for embedding layers. It collects the cardinality of the embedding table and tags it as categorical.

In [18]:
cat_features = ["userId", "movieId"] >> ops.Categorify(dtype="int32", out_path=os.path.join(input_path, "categories"))

The tags for `user`, `userId`, `item` and `itemId` cannot be inferred from the dataset. Therefore, we need to provide them manually during the NVTabular workflow. Actually, the `DLRMModel` does not differentiate between `user` and `item` features. But other architectures, such as the `TwoTowerModel` depends on the `user` and `item` features distinction. We will show how to tag features manually in a NVTabular workflow below.

In [19]:
feats_itemId = cat_features["movieId"] >> ops.TagAsItemID()
feats_userId = cat_features["userId"] >> ops.TagAsUserID()
feats_target = (
    nvt.ColumnSelector(["rating"])
    >> ops.LambdaOp(lambda col: (col > 3).astype("int32"))
    >> ops.AddTags(["binary_classification", "target"])
    >> nvt.ops.Rename(name="rating_binary")
)
output = feats_itemId + feats_userId + feats_target

We fit the workflow to our train set and apply to the valid and test sets.

In [20]:
%%time
train_path = os.path.join(input_path, name, "train.parquet")
valid_path = os.path.join(input_path, name, "valid.parquet")
output_path = os.path.join(input_path, name + "_integration")

workflow_fit_transform(output, train_path, valid_path, output_path)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed i

CPU times: user 856 ms, sys: 123 ms, total: 979 ms
Wall time: 1.01 s


### Training a Recommender Model with Merlin Models

We can load the data as a Merlin Dataset object. The Dataset expect the schema as Protobuf text format (`.pbtxt`) file in the train/valid folder, which NVTabular automatically generates.

In [21]:
train = merlin.io.Dataset(
    os.path.join(input_path, name + "_integration", "train"), engine="parquet"
)
valid = merlin.io.Dataset(
    os.path.join(input_path, name + "_integration", "valid"), engine="parquet"
)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


We can see that the `schema` object contains the features tags and the cardinalities of the categorical features.
As we prepared only a minimal example, our schema has only tree features `movieId`, `userId` and `rating_binary`.|

In [22]:
train.schema.column_names

['movieId', 'userId', 'rating_binary']

In [23]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.start_index,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name
0,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID, Tags.IT...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,/root/merlin-models-data/movielens/categories/...,0.0,159.0,3677.0,0.0,3676.0,movieId
1,userId,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,/root/merlin-models-data/movielens/categories/...,0.0,210.0,6041.0,0.0,6040.0,userId
2,rating_binary,"(Tags.BINARY_CLASSIFICATION, Tags.TARGET)","DType(name='int32', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
valid.schema.column_names

['movieId', 'userId', 'rating_binary']

In [25]:
valid.schema

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.start_index,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name
0,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID, Tags.IT...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,/root/merlin-models-data/movielens/categories/...,0.0,159.0,3677.0,0.0,3676.0,movieId
1,userId,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,0.0,NaN,0.0,/root/merlin-models-data/movielens/categories/...,0.0,210.0,6041.0,0.0,6040.0,userId
2,rating_binary,"(Tags.BINARY_CLASSIFICATION, Tags.TARGET)","DType(name='int32', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here we train our model.
We use `BinaryOutput` because we want to train a binary classification task, and specify which column should be used as target. We could also have used `OutputBlock`, which automatically infers from schema which one target column is based on the tags.
P.s. If there are multiple targets, `OutputBlock` would create a `ModelOutput` for each target column.

In [26]:
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryOutput(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)

model.compile(optimizer="adam")
model.fit(train, batch_size=1024)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


782/782 [==============================] - 13s 11ms/step - loss: 0.6150 - precision: 0.6419 - recall: 0.8827 - binary_accuracy: 0.6490 - auc: 0.6871 - regularization_loss: 0.0000e+00 - loss_batch: 0.6149


Let's run the evaluation on validations set. We use by default typical binary classification metrics -- Precision, Recall, Accuracy and AUC. But you also can provide your own metrics list by setting `BinaryClassificationTask(..., metrics=[])`.

In [27]:
metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


196/196 [==============================] - 2s 6ms/step - loss: 0.5416 - precision: 0.7308 - recall: 0.8160 - binary_accuracy: 0.7221 - auc: 0.7906 - regularization_loss: 0.0000e+00 - loss_batch: 0.5417


In [28]:
metrics

{'loss': 0.5416054129600525,
 'precision': 0.7308200001716614,
 'recall': 0.8160365223884583,
 'binary_accuracy': 0.7221469879150391,
 'auc': 0.7906007766723633,
 'regularization_loss': 0.0,
 'loss_batch': 0.5493170022964478}

In [29]:
model.save(os.path.join("custom_dlrm_1m"))


/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [34]:
import tensorflow as tf
loaded_model = tf.saved_model.load("/content/custom_dlrm_1m")

from merlin.dataloader.tensorflow import Loader




In [43]:
selected_movies = [9, 2, 3, 4, 5]

In [63]:
#selected_movies = st.multiselect("Select 5 movies", options=movie_options, format_func=lambda x: str(x))
movie_options = valid.to_ddf().compute()["movieId"].unique().to_numpy()
if len(selected_movies) == 5:
    # Create a new user with the selected movies
    new_user_id = valid.to_ddf().compute()["userId"].max() + 1

    # Create a new dataset that contains all the possible combinations of the new user and all the movies in your collection
    test_df = pd.DataFrame({
        "userId": [new_user_id] * len(movie_options),
        "movieId": movie_options,
        "rating": [0] * len(movie_options)
    })

    # Add additional rows to the test data for the selected movies with a high rating value
    selected_movies_df = pd.DataFrame({
        "userId": [new_user_id] * len(selected_movies),
        "movieId": selected_movies,
        "rating": [5] * len(selected_movies),
    })
    test_df = pd.concat([test_df, selected_movies_df], ignore_index=True)

    # Save the test data to a parquet file
    test_path = os.path.join(input_path, name + "_integration", "test.parquet")
    test_df.to_parquet(test_path)

    # Load the test data using the merlin.io.Dataset class
    test_ds = merlin.io.Dataset(test_path, engine="parquet")
    loader = Loader(test_ds, batch_size=1)

    # Get the predictions for all the movies for the new user
    predictions = model.predict(loader, verbose=0)

    # Get the top 10 movie recommendations for the new user, excluding the selected movies
    top_10_movie_indices = predictions.argsort()[-15:][::-1]
    top_10_movie_recommendations = []
    for index in top_10_movie_indices:
        movie_id = test_df.iloc[index]["movieId"]
        if movie_id not in selected_movies:
            top_10_movie_recommendations.append(movie_id)
        if len(top_10_movie_recommendations) == 10:
            break

/usr/local/lib/python3.10/dist-packages/merlin/dataloader/tensorflow.py:65: UserWarning: Due to a CUDA memory alignment issue in some Tensorflow operations such as Embedding ops, we recommend that 'batch_size' be at least 16 and also a power of two. Please change 'batch_size' to a number that is a power of two that is greater than or equal to 16.
  warnings.warn(


KeyboardInterrupt: ignored

In [61]:
import numpy as np
unique_values = np.unique(predictions)
unique_values

array([0.5140152], dtype=float32)

In [88]:
# Ask the user to enter their userId
#user_id = st.number_input("Enter your userId", min_value=0, step=1)
user_id = 77
#if st.button("Get recommendations"):
# Get the movies and ratings for the selected user from the validation data
user_data = valid.to_ddf().compute()
user_data = user_data[user_data["userId"] == user_id]
#print(user_data)
    #st.write("Movies and ratings for the selected user:")
    #st.write(user_data[["movieId", "rating"]])
print(user_data[["movieId", "rating_binary"]])
    # Create a new dataset that contains all the possible combinations of the existing user and all the movies in your collection
test_df = pd.DataFrame({
    "userId": [user_id] * len(movie_options),
    "movieId": movie_options,
})

# Save the test data to a parquet file
test_path = os.path.join(input_path, name + "_integration", "test.parquet")
test_df.to_parquet(test_path)

# Load the test data using the merlin.io.Dataset class
test_ds = merlin.io.Dataset(test_path, engine="parquet")
loader = Loader(test_ds, batch_size=1)

# Get the predictions for all the movies for the existing user
predictions = model.predict(loader, verbose=0)


        movieId  rating_binary
195         438              1
1791         90              1
3306        880              1
3436        157              1
4860        602              1
...         ...            ...
196938      240              1
197406      604              1
197854      751              1
198669      575              1
199463      267              0

[171 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/merlin/dataloader/tensorflow.py:65: UserWarning: Due to a CUDA memory alignment issue in some Tensorflow operations such as Embedding ops, we recommend that 'batch_size' be at least 16 and also a power of two. Please change 'batch_size' to a number that is a power of two that is greater than or equal to 16.
  warnings.warn(


In [102]:
# Reading the movies.dat file
movies = pd.read_csv(
    os.path.join(input_path, "ml-1m/movies.dat"),
    sep="::",
    names=["movieId", "title", "genre"],
    engine='python',
    encoding='latin-1'
)

# Convert the user_data["movieId"] CuPy array to a Pandas DataFrame
rated_movies = pd.DataFrame(user_data["movieId"].values.get(), columns=["movieId"])

# Flatten your predictions array and get the top 15 predicted ratings indices in descending order
predictions_flattened = predictions.flatten()
top_15_indices = predictions_flattened.argsort()[-15:][::-1]

# Get the corresponding movieIds for these indices
top_15_movies = test_df.reset_index().iloc[top_15_indices]["movieId"]

# Exclude the movies already rated by the user
recommended_movies = top_15_movies[~top_15_movies.isin(rated_movies["movieId"])]

# Get the top 10 movie recommendations
top_10_movie_recommendations = recommended_movies.head(10)

# Convert the top_10_movie_recommendations to DataFrame
top_10_movie_recommendations_df = pd.DataFrame(top_10_movie_recommendations, columns=['movieId'])

# Merge with the movies DataFrame
top_10_movie_titles = pd.merge(top_10_movie_recommendations_df, movies, on='movieId', how='inner')

# Display the movie titles
print(top_10_movie_titles['title'])


0                      Balto (1995)
1           Cutthroat Island (1995)
2                    Jumanji (1995)
3                 Get Shorty (1995)
4                  Drop Zone (1994)
5          Leaving Las Vegas (1995)
6    American President, The (1995)
7                 To Die For (1995)
8    Moonlight and Valentino (1995)
9           Grumpier Old Men (1995)
Name: title, dtype: object


In [94]:
# Get the movie recommendations for the existing user with predicted ratings greater than 0.5, excluding movies that have already been rated by the user
recommended_movie_indices = np.where(predictions > 0.5)[0]
recommended_movies = []
for index in recommended_movie_indices:
    movie_id = test_df.iloc[index]["movieId"]
    if not np.isin([movie_id], rated_movies["movieId"].values).any():
        recommended_movies.append(movie_id)

In [101]:
top_10_movie_recommendations

44       13
394      15
264       2
285      21
33      227
223      25
387      11
313      45
1464    182
283       3
Name: movieId, dtype: int32

In [93]:
predictions

array([[0.68412226],
       [0.38510266],
       [0.3861965 ],
       ...,
       [0.3496854 ],
       [0.5509841 ],
       [0.5075224 ]], dtype=float32)

In [84]:
test_df.iloc[0:10]

,userId,movieId
0,77,944
1,77,695
2,77,1182
3,77,1608
4,77,1246
5,77,1844
6,77,681
7,77,906
8,77,832
9,77,1931


In [ ]:
loader = Loader(valid, batch_size=1)

pred = model.predict(loader, verbose=0)[:10]

pred

In [31]:
from merlin.datasets.entertainment import get_movielens

# Downloading the MovieLens dataset
data = get_movielens(variant="ml-1m", path=input_path)

# Getting the first Dataset object from the tuple
dataset1 = data[0]

# Converting the Dataset object to a Dask DataFrame
ddf1 = dataset1.to_ddf()

# Computing the result to get a Pandas DataFrame
df1 = ddf1.compute()

# Printing the first 5 rows of the DataFrame
print(df1.head())

# Getting the second Dataset object from the tuple
dataset2 = data[1]

# Converting the Dataset object to a Dask DataFrame
ddf2 = dataset2.to_ddf()

# Computing the result to get a Pandas DataFrame
df2 = ddf2.compute()

# Printing the first 5 rows of the DataFrame
print(df2.head())


   userId  movieId  title     genres  gender  age  occupation  zipcode  \
0      82     1880   1881        [9]       1    3          12      241   
1     328     1322   1325  [3, 9, 5]       1    1           8        8   
2     179       21     21    [2, 11]       1    3           2       12   
3    1834      252    253    [14, 4]       2    2           4      681   
4    1081      446    446    [2, 11]       2    4          20      934   

   TE_age_rating  TE_gender_rating  TE_occupation_rating  TE_zipcode_rating  \
0      -1.058193         -0.558845              1.358930          -0.696975   
1      -0.513785         -0.485006             -0.153052           0.450361   
2      -1.085637         -0.660698             -0.831070           0.117446   
3       0.530947          1.814615             -0.182642           0.380276   
4       0.856190          1.636596              1.306146           0.020537   

   TE_movieId_rating  TE_userId_rating  rating_binary  rating  
0          -2.25

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [ ]:
import os
import pandas as pd
from merlin.dataloader.tensorflow import Loader


# Reading the movies.dat file
movies = pd.read_csv(
    os.path.join(input_path, "ml-1m/movies.dat"),
    sep="::",
    names=["movieId", "title", "genre"],
    engine='python',
    encoding='latin-1'
)

# ratings = pd.read_csv(
#     os.path.join(input_path, "ml-1m/ratings.dat"),
#     sep="::",
#     names=["userId", "movieId", "rating", "timestamp"],
# )

# Creating a Loader object for the valid dataset with batch size 1
loader = Loader(valid, batch_size=1)

# Generating predictions for all batches of data from the valid dataset
predictions = model.predict(loader, verbose=0)

# Converting the valid dataset to a DataFrame
valid_df = valid.to_ddf().compute()



/usr/local/lib/python3.10/dist-packages/merlin/dataloader/tensorflow.py:65: UserWarning: Due to a CUDA memory alignment issue in some Tensorflow operations such as Embedding ops, we recommend that 'batch_size' be at least 16 and also a power of two. Please change 'batch_size' to a number that is a power of two that is greater than or equal to 16.
  warnings.warn(


TypeError: ignored

In [ ]:
valid_df

,movieId,userId,rating_binary,prediction,gender_x,age_x,occupation_x,zip-code_x,title_x,genres_x,gender_y,age_y,occupation_y,zip-code_y,title_y,genres_y
0,767,104,0,0.082940,M,25,12,00926,"Grass Harp, The (1995)",Drama,M,25,12,00926,"Grass Harp, The (1995)",Drama
1,406,954,1,0.402485,F,25,0,21030,Federal Hill (1994),Drama,F,25,0,21030,Federal Hill (1994),Drama
2,406,230,0,0.108031,M,45,1,43210,Federal Hill (1994),Drama,M,45,1,43210,Federal Hill (1994),Drama
3,4,971,1,0.827907,M,25,12,43615,Waiting to Exhale (1995),Comedy|Drama,M,25,12,43615,Waiting to Exhale (1995),Comedy|Drama
4,236,4156,0,0.522660,M,56,20,95123,French Kiss (1995),Comedy|Romance,M,56,20,95123,French Kiss (1995),Comedy|Romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,342,4128,1,0.553437,M,35,6,60025,Muriel's Wedding (1994),Comedy|Romance,M,35,6,60025,Muriel's Wedding (1994),Comedy|Romance
493,1265,1436,0,0.629831,F,50,6,97005,Groundhog Day (1993),Comedy|Romance,F,50,6,97005,Groundhog Day (1993),Comedy|Romance
494,234,189,1,0.610184,M,18,0,60076,Exit to Eden (1994),Comedy,M,18,0,60076,Exit to Eden (1994),Comedy
495,356,2561,1,0.226114,M,25,15,06460,Forrest Gump (1994),Comedy|Romance|War,M,25,15,06460,Forrest Gump (1994),Comedy|Romance|War


In [ ]:
valid_df = valid.to_ddf().compute()
users = pd.read_csv(
    os.path.join(input_path, "ml-1m/users.dat"),
    sep="::",
    names=["userId", "gender", "age", "occupation", "zip-code"],
    engine='python',
    encoding='latin-1'
)

movies = pd.read_csv(
    os.path.join(input_path, "ml-1m/movies.dat"),
    sep="::",
    names=["movieId", "title", "genres"],
    engine='python',
    encoding='latin-1'
)
valid_df = valid_df.to_pandas()
valid_df = valid_df.head(500)

# Flattening the predictions array into a 1-dimensional array
predictions = predictions.ravel()

# Converting the valid dataset to a DataFrame
#valid_df = valid.to_ddf().compute()
print(len(predictions[:500]))
print(len(valid_df))

# Adding the predictions to the valid DataFrame
valid_df['prediction'] = predictions[:500]


# Filtering the rows where the prediction is greater than or equal to 0.5
valid_df = valid_df[valid_df['prediction'] >= 0.5]

# Merging the valid DataFrame with the users DataFrame to add user details
valid_df = pd.merge(valid_df, users, on='userId', suffixes=('', '_users'))

# Merging the valid DataFrame with the movies DataFrame to add movie details
valid_df = pd.merge(valid_df, movies, on='movieId', suffixes=('', '_movies'))

ratings = pd.read_csv(
    os.path.join(input_path, "ml-1m/ratings.dat"),
    sep="::",
    names=["userId", "movieId", "rating", "timestamp"],
    engine='python',
    encoding='latin-1'
)

valid_df = pd.merge(valid_df, ratings, on=['userId', 'movieId'], suffixes=('', '_ratings'))

# Grouping the valid DataFrame by userId
grouped = valid_df.groupby('userId')

for userId, group in grouped:
    # Printing the userId
    print('userId:', userId)

    # Printing the user details
    print('Gender:', group['gender'].iloc[0])
    print('Age:', group['age'].iloc[0])
    print('Occupation:', group['occupation'].iloc[0])
    print('Zip-code:', group['zip-code'].iloc[0])

    # Printing the movie details, predictions and ratings
    for i, row in group.head(500).iterrows():
        print('Movie title:', row['title'])
        print('Prediction:', row['prediction'])
        print('Rating:', row['rating'])

    # Printing a separator
    print('-' * 20)



# # Iterating over the groups
# for userId, group in grouped:
#     # Printing the userId
#     print('userId:', userId)

#     # Printing the user details
#     print('Gender:', group['gender'].iloc[0])
#     print('Age:', group['age'].iloc[0])
#     print('Occupation:', group['occupation'].iloc[0])
#     print('Zip-code:', group['zip-code'].iloc[0])

#     # Printing the movie details and predictions
#     for i, row in group.head(500).iterrows():
#         print('Movie title:', row['title'])
#         print('Prediction:', row['prediction'])

#     # Printing a separator
#     print('-' * 20)

500
500
userId: 22
Gender: M
Age: 18
Occupation: 15
Zip-code: 53706
Movie title: Wag the Dog (1997)
Prediction: 0.5736192464828491
Rating: 4
--------------------
userId: 40
Gender: M
Age: 45
Occupation: 0
Zip-code: 10543
Movie title: Escape from New York (1981)
Prediction: 0.6885989904403687
Rating: 3
--------------------
userId: 74
Gender: M
Age: 35
Occupation: 14
Zip-code: 94530
Movie title: Usual Suspects, The (1995)
Prediction: 0.7448956370353699
Rating: 5
--------------------
userId: 195
Gender: M
Age: 25
Occupation: 12
Zip-code: 10458
Movie title: M*A*S*H (1970)
Prediction: 0.767417311668396
Rating: 4
--------------------
userId: 329
Gender: M
Age: 35
Occupation: 7
Zip-code: 02115
Movie title: Shining, The (1980)
Prediction: 0.7629075646400452
Rating: 4
--------------------
userId: 411
Gender: F
Age: 45
Occupation: 1
Zip-code: 43214
Movie title: What's Eating Gilbert Grape (1993)
Prediction: 0.5827045440673828
Rating: 4
--------------------
userId: 438
Gender: M
Age: 18
Occupatio

In [ ]:
# Iterating over the rows of the valid DataFrame
for i, row in valid_df.to_pandas().iterrows():
    # Getting the prediction for the current row
    prediction = predictions[i]

    # Checking if the prediction is greater than or equal to 0.5
    # Checking if the prediction is greater than or equal to 0.5
    if prediction >= 0.5:
        # Getting the userId and movieId of the current row
        userId = row['userId']
        movieId = row['movieId']

        # Checking if there is at least one row in the movies DataFrame where movieId is equal to movieId
        if (movies['movieId'] == movieId).any():
            # Getting the title of the movie
            title = movies[movies['movieId'] == movieId]['title'].iloc[0]

            print('userId:', userId, 'Movie title:', title, 'Prediction:', prediction)
        else:
            print('No movie found with movieId:', movieId)



Streaming output truncated to the last 5000 lines.
userId: 428 Movie title: West Side Story (1961) Prediction: [0.81679165]
userId: 309 Movie title: Shallow Grave (1994) Prediction: [0.7067399]
userId: 516 Movie title: Silence of the Palace, The (Saimt el Qusur) (1994) Prediction: [0.7847102]
userId: 722 Movie title: Margaret's Museum (1995) Prediction: [0.88536483]
userId: 3771 Movie title: Taxi Driver (1976) Prediction: [0.865758]
userId: 760 Movie title: Geronimo: An American Legend (1993) Prediction: [0.5389528]
userId: 2719 Movie title: Black Beauty (1994) Prediction: [0.52720755]
userId: 3120 Movie title: Last of the High Kings, The (a.k.a. Summer Fling) (1996) Prediction: [0.55123866]
userId: 2973 Movie title: Apollo 13 (1995) Prediction: [0.5514062]
userId: 1141 Movie title: Oliver & Company (1988) Prediction: [0.5533249]
userId: 455 Movie title: Jury Duty (1995) Prediction: [0.80760986]
userId: 1402 Movie title: Of Love and Shadows (1994) Prediction: [0.5922649]
userId: 837 Mo